In [3]:
import numpy as np
import K_Means_Cluster as clusterMod
from matplotlib import pylab as plt
import time
import os
import sys
from matplotlib import style 
style.use('ggplot')
colors = 10*["g","r","c","b","k"]

In [4]:
def locateFiles(): #locates the files in the folder and assigns them to a directory
    directory = {}
    listdir = os.listdir("Raw_Data_with_hdr") # access the folder
    
    for file in listdir:
        key = find_between(file,"raw_", "_")
        try:
            if key is "" or not(file.endswith(".raw") or file.endswith(".hdr") or file.endswith('_rd')):
                raise ValueError (file, " :Either the name of this file is not formatted correctly or the type is incorrect")
        except ValueError as exc:
            print(exc)
            print(" ")
            print("Please refer to the readme")
            sys.exit()

        if key in directory:
            if file.endswith('.raw') or file.endswith('_rd'):
                directory[key] = [file, directory[key][0]]
            else:
                directory[key].append(file)
            try:
                if len(directory[key]) > 2:
                    raise ValueError (directory[key], " :there are more than two files with the same name")
            except ValueError as exc:
                print(exc)
                print(" ")
                print("Please refer to the readme")
                sys.exit()
        else:
            directory[key] = [file]
    return directory

In [5]:
def find_between( s, first, last ): #helper method to isolate numbers in a list
        try:
            start = s.index( first ) + len( first )
            end = s.index( last, start )
            return s[start:end]
        except ValueError:
            return ""

In [6]:
def readFileFromDirectory(key, hdr): #reads files from the file folder using the directory names
    #hdr = 1 to designate we are looking for header file, 0 to just read raw file
    
    script_dir =  os.path.dirname(os.path.realpath('__file__'))
    rel_path = ('Raw_Data_with_hdr\\') + (directory[key][0])
    if hdr is 1:
        rel_path = ('Raw_Data_with_hdr\\') + (directory[key][1])
        abs_file_path = os.path.join(script_dir, rel_path)
        with open(abs_file_path, 'r') as file: 
            headerFile = file.read().splitlines()
        return headerFile
    else:
        abs_file_path = os.path.join(script_dir, rel_path)
        raw_data = np.fromfile(abs_file_path, dtype='>H', sep="")
        return raw_data

In [7]:
def processRawData(length, width, num_bands, keys):
#processing the raw data into a more readable format
#length(rows) X width(col) X bands(depth),, keys = file number we are looking at
    raw_data = readFileFromDirectory(keys,0)

    processed = np.empty(shape=(length,width, num_bands))
    process_start_time = time.time()
    counter = 0
    for x in range(num_bands):
        for j in range(length):
            for i in range(width):
                if counter >= raw_data.size:
                    break
                g = raw_data[counter]
                processed[j][i][x] = g
                counter += 1
    process_total_time = time.time() - process_start_time
    print("Time to Process: ", process_total_time, "seconds")
    return processed

In [13]:
def writeToFile(clusterCords, key):
#input clusterCords = (dictionary object. dictkeys = cluster number, items = array in form [row, column])
#      key = file number we are working with
    remap = np.zeros((length,width))
    for group in clusterCords:
        for featureset in clusterCords[group]:
            remap[featureset[0]][featureset[1]] = str(group)
    title = str(key) + "_file_" + "output" + ".txt"
    np.savetxt(title, remap, fmt='%.2f')

In [10]:
#if __name__ == '__main__':
directory = locateFiles()
#for keys in directory:
keys = '0'
if keys is '0':
    header = readFileFromDirectory(keys, 1)    
    length = int(header[3].split(" = ",1)[1])
    width = int(header[2].split(" = ",1)[1])
    bands = int(header[4].split(" = ",1)[1])
    
    
    processed = processRawData(length, width, bands, keys)
    
    cluster_start_time = time.time()
    clf = clusterMod.K_Means(k = 2, max_width = min(length, width), max_iter = 10)
    clf.fit(processed)
    cluster_total_time = time.time() - cluster_start_time
    
    print("Time to Cluster: ", cluster_total_time, "seconds")
    
    writeToFile(clf.cluster_cords, keys) 
    #write to the coordinates with their group to an output file titled (file number)output.txt

Time to Process:  37.12942600250244 seconds
Time to Cluster:  336.6086800098419 seconds
